<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Working_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! apt-get install libboost-python1.71-dev openbabel libopenbabel-dev python-dev zlib1g-dev libeigen3-dev libgsl-dev
!rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!more pyMol2.cpp | grep VectorInt
!make clean
!make -j4

In [2]:
!ls *.so

pyConformer.so	pyEnergy2.so  pyGaussian.so  pyMol2.so	  pyRAND.so
pyCOORD_MC.so	pyFindHB.so   pyGrid.so      pyPARSER.so  pyWRITER.so


In [3]:
%%capture
#!make clean
#!make
#!ls *.so

In [4]:
from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *

In [ ]:
Input = PARSER()
Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")

for i in range(len(list(rec.residue_pointer))-1):
  HB.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)

HB.find_ligandHB("test/lig.mol2.gz", lig)

com = Coord.compute_com(lig)

Input.write_grids = True;
Grids = Grid(Input, Writer, rec, com)